# Example 3.1: Piecewise linear bond slip law 
This sheet demonstrates the implemented finite element model that shall be used later on to study the debonding behavior of different materials. You can modify the parameters of the bond slip law to study its effect on the structural behavior.

In [ ]:
%reset -f
%matplotlib inline
import matplotlib.pyplot as plt
from bmcs.api import PullOutModel
import numpy as np

## Define the material parameters

In [ ]:
w_max = 0.1  # [mm]
d = 16.0  # [mm]
E_f = 210000  # [MPa]
E_m = 28000  # [MPa]
A_f = (d / 2.)**2 * np.pi  # [mm^2]
P_b = d * np.pi
A_m = 10 * d * 10 * d  # [mm^2]
L_x = 5 * d

## Construct the finite element model of the Pullout
The class PullOutModel can be configured using different types of material models, number of elements and loading scenarios. In the present case, lets use it in a simple way applying a linear kind of bind-slip law up to the pullout displacment $w=0.1\;\mathrm{mm}$ 

In [ ]:
pm = PullOutModel(mats_eval_type='multilinear',
                  n_e_x=50, w_max=w_max)
pm.tline.step=0.02
pm.tloop.verbose=False
pm.loading_scenario.loading_type = 'monotonic'
pm.mats_eval.trait_set(E_m=E_m, E_f=E_f,
                       s_data='0, 0.01, 0.1',
                       tau_data='0, 1, 1.1')
pm.mats_eval.update_bs_law = True
pm.cross_section.trait_set(A_m=A_m, P_b=P_b, A_f=A_f)
pm.geometry.L_x = L_x

## Bond-slip law
The material model of the pullout model is accessible via the attribute mats_eval. Let us check if the bond slip law has been configured correctly by plotting it.

In [ ]:
pm.mats_eval.bs_law.plot(plt, color='green')
plt.xlabel('slip [mm]')
plt.ylabel('tau [MPa]')

## Run the solver
The model will use the defined discretization, assemble the stiffness matrix of the system and solve it by returning the diplacement field.

In [ ]:
pm.run()

## Postprocessing
Let us now access the fields obtained by the solver

### Pullout response

In [ ]:
P, w0, wL = pm.get_Pw_t()
plt.plot(wL,P)
plt.plot(w0,P)

### Displacement fields

In [ ]:
pm.plot_u_p(plt.axes(),1)

### Slip field

In [ ]:
pm.plot_s(plt.axes(),1)

### Shear flow $T = \tau p$

In [ ]:
pm.plot_sf(plt.axes(),0.1)

### Strain fields

In [ ]:
pm.plot_eps_p(plt.axes(),0.9999)

### Force fields $F = \sigma_c A_c$

In [ ]:
pm.plot_sig_p(plt.axes(),0.02)
pm.plot_sig_p(plt.axes(),0.999)

# Questions and tasks

### 1 How many elements are required to get the accaptable accuracy?
Test if the solution changes if the number of elements gets changed

### 2 Why are the stresses and strains constant within a finite element?
If displacements are linear within an element - the strains are their derivatives ... 

### 3 At which distance from the loaded point do we get zero slip?
Recalling that we assumed $a$ finite for the examples with constant bond slip law. Can we find such a distance in case of the linear bond-slip law?